In [1]:
import numpy as np
import json
import csv
import pandas as pd
import re
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

In [7]:
year_list = [str(year) for year in range(1978,2019 + 1)]

data = []
with open('../../nominate_movie_meta_data.json', 'r', encoding = 'utf-8') as f:
    json_dict = json.load(f)
    for year in year_list:
        data += json_dict[year]
        
data = pd.DataFrame(data).set_index('id')

def dict2list_other_nominate(series):
    l = []
    other_nominates = series['other_nominates']
    for each in other_nominates:
        if each['prized'] == 1:
            l.append(each['award'])
    return pd.Series([l])

data['other_nominates'] = data.apply(dict2list_other_nominate, axis = 'columns')

data['production_studio'] = data['production_studio'].map(lambda each: re.split(r'[、＝=]',each))

mlb = MultiLabelBinarizer()
onehot = mlb.fit_transform(data['other_nominates'])
columns = mlb.classes_
nomination_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################

performers = []
data['performers'].map(lambda each: performers.extend(each))

count = Counter(performers)
countSeries = pd.Series(count)
selected_performers = countSeries[countSeries > 9].index ######10回以上受賞作品に出演

data['selected_performers'] = data['performers'].map(lambda each: list( set(each) & set(selected_performers) ))

onehot = mlb.fit_transform(data['selected_performers'])
columns = mlb.classes_
selected_performers_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################10回以上出た出演者のonehot


directors = []
data['director'].map(lambda each: directors.extend(each))
    
count = Counter(directors)
countSeries = pd.Series(count)
selected_directors = countSeries[countSeries > 3].index

data['selected_directors'] = data['director'].map(lambda each: list( set(each) & set(selected_directors) ))

onehot = mlb.fit_transform(data['selected_directors'])
columns = mlb.classes_
selected_directors_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################4回以上出た監督のonehot

studio = []
data['production_studio'].map(lambda each: studio.extend(each))

count = Counter(studio)
countSeries = pd.Series(count)
selected_studios = countSeries[countSeries > 20].index ########################### 20回以上受賞

data['selected_studio'] = data['production_studio'].map(lambda each: list( set(each) & set(selected_studios) ))

onehot = mlb.fit_transform(data['selected_studio'])
columns = mlb.classes_
selected_studio_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################4回以上出た監督のonehot

scriptwriters = []
data['scriptwriter'].map(lambda each: scriptwriters.extend(each))

count = Counter(scriptwriters)
countSeries = pd.Series(count)
selected_scriptwriters = countSeries[countSeries > 2].index

data['selected_scriptwriter'] = data['scriptwriter'].map(lambda each: list( set(each) & set(selected_scriptwriters) ))

onehot = mlb.fit_transform(data['selected_scriptwriter'])
columns = mlb.classes_
selected_scriptwriter_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #############

In [15]:
with open('2020.json', 'r', encoding = 'utf-8') as f:
    json_dict = json.load(f)

target = pd.DataFrame(json_dict['2020'])



def dict2list_other_nominate(series):
    l = []
    other_nominates = series.get('other_nominates')
    if other_nominates is not None:
        for each in other_nominates:
            if each['prized'] == 1:
                l.append(each['award'])
    return pd.Series([l])

target['other_nominates'] = data.apply(dict2list_other_nominate, axis = 'columns')



data['production_studio'] = data['production_studio'].map(lambda each: re.split(r'[、＝=]',each))

mlb = MultiLabelBinarizer()
onehot = mlb.fit_transform(data['other_nominates'])
columns = mlb.classes_
nomination_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################

performers = []
data['performers'].map(lambda each: performers.extend(each))

count = Counter(performers)
countSeries = pd.Series(count)
selected_performers = countSeries[countSeries > 9].index ######10回以上受賞作品に出演

data['selected_performers'] = data['performers'].map(lambda each: list( set(each) & set(selected_performers) ))

onehot = mlb.fit_transform(data['selected_performers'])
columns = mlb.classes_
selected_performers_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################10回以上出た出演者のonehot


directors = []
data['director'].map(lambda each: directors.extend(each))
    
count = Counter(directors)
countSeries = pd.Series(count)
selected_directors = countSeries[countSeries > 3].index

data['selected_directors'] = data['director'].map(lambda each: list( set(each) & set(selected_directors) ))

onehot = mlb.fit_transform(data['selected_directors'])
columns = mlb.classes_
selected_directors_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################4回以上出た監督のonehot

studio = []
data['production_studio'].map(lambda each: studio.extend(each))

count = Counter(studio)
countSeries = pd.Series(count)
selected_studios = countSeries[countSeries > 20].index ########################### 20回以上受賞

data['selected_studio'] = data['production_studio'].map(lambda each: list( set(each) & set(selected_studios) ))

onehot = mlb.fit_transform(data['selected_studio'])
columns = mlb.classes_
selected_studio_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #####################4回以上出た監督のonehot

scriptwriters = []
data['scriptwriter'].map(lambda each: scriptwriters.extend(each))

count = Counter(scriptwriters)
countSeries = pd.Series(count)
selected_scriptwriters = countSeries[countSeries > 2].index

data['selected_scriptwriter'] = data['scriptwriter'].map(lambda each: list( set(each) & set(selected_scriptwriters) ))

onehot = mlb.fit_transform(data['selected_scriptwriter'])
columns = mlb.classes_
selected_scriptwriter_onehot = pd.DataFrame(onehot, columns = columns, index = data.index) #############

In [33]:
{
    'title': 'dammy',
    'other_nominates': list(nomination_onehot.columns),
    'screen_time': 893,
    'year': 2999,
    'prize: 1,
    'selected_performers': list(selected_performers_onehot.columns),
    'selected_directors': list(selected_
}

{'title': 'dammy',
 'other_nominates': ['blue_ribbon_award',
  'golden_gross',
  'hochi_eigashou',
  'kinejun_best_ten',
  'mainichi_film_award',
  'nikkan_sports']}

In [36]:
data

,director,scriptwriter,performers,screen_time,production_studio,title,year,prize,other_nominates,selected_performers,selected_directors,selected_studio,selected_scriptwriter
id,,,,,,,,,,,,,
1,[山田洋次],"[山田洋次, 朝間義隆]","[高倉健, 倍賞千恵子, 桃井かおり, 武田鉄矢, 渥美清]",108,[松竹],幸福の黄色いハンカチ,1978,1,"[hochi_eigashou, blue_ribbon_award]",[高倉健],[山田洋次],[松竹],"[朝間義隆, 山田洋次]"
2,"[浦山桐郎, 橋本幸治]","[早坂暁, 浦山桐郎]","[田中健, 大竹しのぶ, いしだあゆみ, 高橋悦史, 伊東辰夫, 高瀬春奈, 高山彰, 梅宮...",161,[東宝],青春の門 自立篇（1977）,1978,0,[],[大竹しのぶ],[],[東宝],[早坂暁]
3,"[新藤兼人, 小松崎和男]",[新藤兼人],"[高橋竹山, 林隆三, 乙羽信子, 金井大, 倍賞美津子, 島村佳江, 観世栄夫, 根岸明美...",125,[独立映画センター],竹山ひとり旅,1978,0,[],[倍賞美津子],[],[],[新藤兼人]
4,"[森谷司郎, 神山征二郎]",[橋本忍],"[島田正吾, 大滝秀治, 高倉健, 丹波哲郎, 藤岡琢也, 浜田晃, 加藤健一, 江幡連, ...",169,[東宝],八甲田山,1978,0,[],"[大滝秀治, 神山繁, 高倉健, 丹波哲郎, 緒形拳]",[神山征二郎],[東宝],[]
5,"[篠田正浩, 永井正夫]","[長谷部慶治, 篠田正浩]","[岩下志麻, 原田芳雄, 奈良岡朋子, 神保共子, 横山リエ, 宮沢亜古, 中村恵子, 殿山...",117,[東宝],はなれ瞽女おりん,1978,0,[],"[樹木希林, 西田敏行, 浜村純]",[篠田正浩],[東宝],[]
6,"[野村芳太郎, 大嶺俊順]",[新藤兼人],"[永島敏行, 松坂慶子, 大竹しのぶ, 佐分利信, 中野誠也, 磯部勉, 芦田伸介, 丹波哲...",138,[松竹],事件,1979,1,[mainichi_film_award],"[大竹しのぶ, 丹波哲郎, 永島敏行]",[],[松竹],[新藤兼人]
7,[大島渚],[],"[長谷川真砂美, 伊佐山ひろ子, 佐藤慶, 殿山泰司, 田村高廣, 吉行和子, 藤竜也, 杉...",108,[],愛の亡霊,1979,0,[],"[吉行和子, 田村高廣]",[],[],[]
8,"[野村芳太郎, 松原信吾]",[井手雅人],"[岩下志麻, 緒形拳, 岩瀬浩規, 吉沢美幸, 石井旬, 蟹江敬三, 穂積隆信, 大滝秀治,...",110,[松竹],鬼畜,1979,0,[],"[大竹しのぶ, 田中邦衛, 大滝秀治, 浜村純, 緒形拳]",[],[松竹],[]
9,"[東陽一, 青池憲司]",[寺山修司],"[永島敏行, 吉田次昭, 森下愛子, 志方亜紀子, 島倉千代子, 内藤武敏, 峰岸徹, 片桐...",103,[ATG],サード,1979,0,"[hochi_eigashou, blue_ribbon_award, kinejun_be...",[永島敏行],[],[],[]
